# HackerEarth Machine Learning Challenge: Exhibit A(rt) 

**Points:**
* Trying to learn Machine Learning by participating in the competitions.
* Achieved accuracy of 94%, and trying to further improve it.
* Beginner in Machine Leraning, so may be there will be naive mistake, **feedbacks are most welcome**.
* Used feature_engine library. Install it on kaggle using `!pip install feature_engine`

In [ ]:
!pip install feature_engine

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.display.max_columns = None

import seaborn as sns
import feature_engine.transformation as vt #Please install #!pip install feature_engine


In [ ]:
df_train =  pd.read_csv('/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/train.csv')

# Function Definitions

In [ ]:
float_var_list = ['Artist Reputation',
 'Height',
 'Width',
 'Weight',
 'Price Of Sculpture',
 'Base Shipping Price']

Artist_Reputation_mean = df_train['Artist Reputation'].mean()
Height_mean = df_train['Height'].mean()
Width_mean = df_train['Width'].mean()
Weight_median = df_train['Weight'].mean()

df_train_copy = df_train.copy(deep=True)
df_train_copy['State_code'] = df_train_copy['Customer Location'].str.split(',', expand=True)[1].str.slice(0, 3)


def create_dummies_columns(df, column_name, prefix_str):
    temp_df  =  pd.get_dummies(df[column_name], prefix=prefix_str, drop_first=True)
    df.drop([column_name],  axis = 1, inplace=True)
    df = pd.concat([df, temp_df], axis = 1)
    return df

def prepare_data(df):
    df.drop(['Customer Id', 'Artist Name'], axis = 1, inplace=True)
    df['Artist Reputation_NA'] = np.where(df['Artist Reputation'].isnull(), 1, 0)
    df['Artist Reputation'].fillna(Artist_Reputation_mean, inplace = True)
    
    df['Transport_NA'] = np.where(df['Transport'].isnull(), 1, 0)
    df['Transport'].fillna(df['Transport'].mode()[0], inplace = True)
    
    df['Remote_Location_NA'] = np.where(df['Remote Location'].isnull(), 1, 0)
    df['Remote Location'].fillna(df['Remote Location'].mode()[0], inplace = True)
    
    df['Height_NA'] = np.where(df['Height'].isnull(), 1, 0)
    df['Height'].fillna(Height_mean, inplace = True)
    df['Height'] = np.exp(df['Height'])
    
    
    df['Width_NA'] = np.where(df['Width'].isnull(), 1, 0)
    df['Width'].fillna(Width_mean, inplace = True)
    df['Width'] = np.log(df['Width'])
    
    df['Weight_NA'] = np.where(df['Weight'].isnull(),1,0)
    df['Weight'].fillna(Weight_median, inplace = True)
    df['Weight'] = np.log(df['Weight'])
    
    df['Material_NA'] = np.where(df['Material'].isnull(), 1, 0)
    df['Material'].fillna('NA', inplace = True)
    #Below code is to fill Random in Material
    #random_sample_Material = df_train_copy['Material'].dropna().sample(df['Material'].isnull().sum(),
    #                                                                   random_state=
    #                                                                   np.log(np.abs(df_train_copy['Cost'])).astype(int))
    
    #random_sample_Material.index = df[df['Material'].isnull()].index
    #df.loc[df['Material'].isnull(), 'Material'] = random_sample_Material
    
    df['Price Of Sculpture'] = np.log(df['Price Of Sculpture'])

    df['Delivery Date'] = pd.to_datetime(df['Delivery Date'], format='%m/%d/%y')
    df['Scheduled Date'] = pd.to_datetime(df['Scheduled Date'], format='%m/%d/%y')
    df['del_date_sch_date_diff'] = (df['Delivery Date'] - df['Scheduled Date']).dt.days
    df['del_date_sch_date_diff'] = np.abs(df['del_date_sch_date_diff'])
    
    df.drop(['Delivery Date', 'Scheduled Date'], axis = 1, inplace=True)
    
    df['city'] = df['Customer Location'].str.split(',', expand=True)[0]
    df['State_code'] = df['Customer Location'].str.split(',', expand=True)[1].str.slice(0, 3)
    df['pin'] = df['Customer Location'].str.split(',', expand=True)[1].str.split(' ', expand=True)[2]
    
    city_others = df[df['State_code'].isna()]['Customer Location'].str.split(' ', expand=True)[0]
    city_others.index = df[df['State_code'].isnull()].index
    
    State_code_others = df[df['State_code'].isna()]['Customer Location'].str.split(' ', expand=True)[1]
    State_code_others.index = df[df['State_code'].isnull()].index
    
    pin_others = df[df['pin'].isna()]['Customer Location'].str.split(' ', expand=True)[2]
    pin_others.index = df[df['pin'].isnull()].index
    
    
    df.loc[df['State_code'].isnull(), 'city'] = city_others
    df.loc[df['State_code'].isnull(), 'State_code'] = State_code_others
    df.loc[df['pin'].isnull(), 'pin'] = pin_others
    
    #df['pin'] = df['pin'].astype(int)
    
    df.drop(['Customer Location'], axis = 1, inplace=True)
    df.drop(['city', 'pin'], axis = 1, inplace=True)    
    return df

# function to create histogram, Q-Q plot and
# boxplot

import scipy.stats as stats


def diagnostic_plots(df, variable):
    # function takes a dataframe (df) and
    # the variable of interest as arguments

    # define figure size
    plt.figure(figsize=(16, 4))

    # histogram
    plt.subplot(1, 3, 1)
    sns.histplot(df[variable], bins=30)
    plt.title('Histogram')

    # Q-Q plot
    plt.subplot(1, 3, 2)
    stats.probplot(df[variable], dist="norm", plot=plt)
    plt.ylabel('RM quantiles')

    # boxplot
    plt.subplot(1, 3, 3)
    sns.boxplot(y=df[variable])
    plt.title('Boxplot')

    plt.show()

In [ ]:
df_train.head(3)

# Float columns Graph

In [ ]:
for x in float_var_list:
   diagnostic_plots(df_train, x)

In [ ]:
df_train = prepare_data(df_train)

In [ ]:
df_train.head(3)

# Float columns graph after preparing data

In [ ]:
for x in float_var_list:
    diagnostic_plots(df_train, x)

In [ ]:
df_train_X = df_train.drop(['Cost'], axis = 1)
df_train_y = np.log(np.abs(df_train['Cost']))

from feature_engine.encoding import OrdinalEncoder
ordinal_enc = OrdinalEncoder(
    # NOTE that we indicate ordered in the encoding_method, otherwise it assings numbers arbitrarily
    #encoding_method='ordered',
    encoding_method='arbitrary',
    variables=['Material', 'State_code'])
#ordinal_enc.fit(df_train_X, df_train_y)
ordinal_enc.fit(df_train_X)
df_train_X = ordinal_enc.transform(df_train_X)

from feature_engine.encoding import OneHotEncoder
ohe_enc = OneHotEncoder(top_categories=None) 
ohe_enc.fit(df_train_X)
df_train_X = ohe_enc.transform(df_train_X)


In [ ]:
df_train_X.head(3)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_train_X, df_train_y, test_size=0.2)


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor 

from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators=500, learning_rate=0.1, max_depth=8, random_state=42
                                ,loss='ls', min_samples_split=5
                               )
# train the model
reg.fit(X_train, y_train)

print(reg.score(X_train, y_train))
print(reg.score(X_test, y_test))

In [ ]:
param_grid = [
{'n_estimators':  [x for x in range(50,500, 20)], 
 'learning_rate' : [0.1, 0.2, 0.3, 0.4, 0.5],
 'max_depth': [x for x in range(5,20)], 
 'min_samples_split': [x for x in range(2,10)]}
]

In [ ]:
reg.fit(df_train_X, df_train_y)

In [ ]:
df_test =  pd.read_csv('/kaggle/input/hackerearth-machine-learning-exhibit-art/dataset/test.csv')
sample_submission = df_test[['Customer Id']]
df_test = prepare_data(df_test)


In [ ]:
df_test = ordinal_enc.transform(df_test)
df_test = ohe_enc.transform(df_test)
print(df_train_X.shape)
print(df_test.shape)
X_test = df_test
y_predicted = reg.predict(X_test)

In [ ]:
sample_submission['Cost'] = np.exp(y_predicted)
sample_submission.to_csv('sample_submission.csv', index=False)